In [1]:
import os
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
sns.set()
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

In [2]:
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/9xkhgts2s6-1.zip

--2021-09-26 15:23:07--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/9xkhgts2s6-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.98.32
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.98.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3745528792 (3.5G) [application/octet-stream]
Saving to: ‘9xkhgts2s6-1.zip’

9xkhgts2s6-1.zip    100%[===================>]   3.49G  27.8MB/s    in 2m 11s  

2021-09-26 15:25:19 (27.2 MB/s) - ‘9xkhgts2s6-1.zip’ saved [3745528792/3745528792]



In [3]:
%%capture
!unzip 9xkhgts2s6-1.zip

In [4]:
!mkdir Dataset

In [18]:
image_directory='Dataset/'

no_covid_images=os.listdir(image_directory+ 'Normal/')
yes_covid_images=os.listdir(image_directory+ 'COVID-19/')

dataset=[]
label=[]

INPUT_SIZE=64

In [19]:
for i , image_name in tqdm(enumerate(no_covid_images)):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'Normal/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i , image_name in tqdm(enumerate(yes_covid_images)):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'COVID-19/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

3270it [02:11, 24.96it/s]
1281it [00:59, 21.41it/s]


In [20]:
dataset=np.array(dataset)
label=np.array(label)

In [23]:
seed = 7
cvscores = []


for i in range(1,11):


  x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.1, shuffle=True)

  x_train=tf.keras.utils.normalize(x_train, axis=1)
  x_test=tf.keras.utils.normalize(x_test, axis=1)

  y_train=tf.keras.utils.to_categorical(y_train , num_classes=2)
  y_test=tf.keras.utils.to_categorical(y_test , num_classes=2)
  
  # create model
  model=Sequential()

  model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(Flatten())
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(2))
  model.add(Activation('softmax'))
  
  
  # Compile model
  model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
  
  print("Fold ",i," :")
  # Fit the model
  model.fit(x_train, y_train, batch_size=32, verbose=0, epochs=100, validation_data=(x_test, y_test), shuffle=False)
  
  print(" Evaluating ..... ")
  # evaluate the model
  scores = model.evaluate(x_test, y_test, batch_size=32)
  
  # final Scores
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)
  print("\n\n")


print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Fold  1  :
 Evaluating ..... 
15/15 [==============================] - 0s 5ms/step - loss: 0.0276 - accuracy: 0.9912
accuracy: 99.12%



Fold  2  :
 Evaluating ..... 
15/15 [==============================] - 0s 5ms/step - loss: 0.0034 - accuracy: 0.9978
accuracy: 99.78%



Fold  3  :
 Evaluating ..... 
15/15 [==============================] - 0s 6ms/step - loss: 0.0753 - accuracy: 0.9934
accuracy: 99.34%



Fold  4  :
 Evaluating ..... 
15/15 [==============================] - 0s 5ms/step - loss: 1.3723e-04 - accuracy: 1.0000
accuracy: 100.00%



Fold  5  :
 Evaluating ..... 
15/15 [==============================] - 0s 5ms/step - loss: 0.0774 - accuracy: 0.9912
accuracy: 99.12%



Fold  6  :
 Evaluating ..... 
15/15 [==============================] - 0s 6ms/step - loss: 0.1111 - accuracy: 0.9890
accuracy: 98.90%



Fold  7  :
 Evaluating ..... 
15/15 [==============================] - 0s 6ms/step - loss: 0.0301 - accuracy: 0.9956
accuracy: 99.56%



Fold  8  :
 Evaluating ..... 
15/15 

In [24]:
print(cvscores)

[99.12280440330505, 99.78070259094238, 99.34210777282715, 100.0, 99.12280440330505, 98.90350699424744, 99.56140518188477, 100.0, 99.56140518188477, 99.34210777282715]
